# Results

> This handles results of a GPFA/SimpleGP Imputation with plotting and metrics

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
#| default_exp results

In [ ]:
#| export
from meteo_imp.gpfa.learner import *
from meteo_imp.data_preparation import *
from meteo_imp.gpfa.imputation import *
from meteo_imp.utils import *

import torch

import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import mean_squared_error, r2_score

from fastcore.foundation import patch, patch_to
from fastcore.meta import delegates
from fastcore.basics import store_attr, listify
from fastcore.test import test_close
from itertools import zip_longest
from fastcore.dispatch import typedispatch

import matplotlib.pyplot as plt
import altair as alt
from altair import datum

from typing import Collection


In [ ]:
#| export
class ImputationResult:
    def __init__(self,
                 data_imputed, #imputed data in tidy format
                 data_complete, # complete data in tidy format
                 model_info, # learner for parameters display
                 units = None, # units for plots
                 metrics_all_data = True # Compute metrics only for gap or for all data?
                ):
        store_attr()

    def __repr__(self: GPFAImputation):
        return f"""Imputation Result:
        N obs: {self.data_imputed.time.unique().shape[0]}"""

    def __str__(self: GPFAImputation):
        return self.__repr__()

In [ ]:
np.array([5, 3])

array([5, 3])

constructor methods from `GPFAImputation` and `GPFAImputationResults`

In [ ]:
#| export
@patch
def to_result(self: GPFAImputation, data_complete, units=None):
    var_names = self.data.columns
    return ImputationResult(self.impute(add_time=True), data_complete, self.learner.model.get_info(var_names), units, metrics_all_data=False)

In [ ]:
#| export
@patch
def to_result(self: GPFAImputationExplorer, data_complete, units=None):
    var_names = self.data.columns
    return ImputationResult(self.predict(), data_complete, self.learner.model.get_info(var_names), units)

In [ ]:
reset_seed()
fd = GPFADataTest.generate(2, 10, Lambda=[1,2.]).add_random_missing()

imp = GPFAImputation(fd.data)
imp_exp = GPFAImputationExplorer(fd.data)

NameError: name 'torch' is not defined

In [ ]:
fd.data

In [ ]:
res = imp.to_result(fd.data_compl_tidy, units = {'x0': 'x0 units', 'x1': 'x1 unitssss'})
res_exp = imp_exp.to_result(fd.data_compl_tidy, units = {'x0': 'x0 units', 'x1': 'x1 unitssss'})

### Metrics

In [ ]:
#| export
@patch
def compute_metric(self: ImputationResult,
                   metric,  # function that takes as argument true and pred and returns the metric
                   metric_name = 'metric',
                   ):
    df = pd.merge(self.data_imputed, self.data_complete, on = ['time','variable'])
    
    vars = []
    
    for var in df.variable.unique():
        mask = (df.variable == var) & (df.is_missing == True) if not self.metrics_all_data else df.variable == var
        
        df_var = df[mask]
        vars.append({'variable': var,
                      metric_name: metric(df_var['value'], df_var['mean']) if len(df_var) > 0 else None})
    
    return pd.DataFrame(vars)

In [ ]:
#| exports
@patch
def rmse(self: ImputationResult):
    rmse = self.compute_metric(lambda x, y: np.sqrt(mean_squared_error(x,y)), "rmse")
    if self.units: rmse = rmse.assign(units= self.units.values())
    return rmse

In [ ]:
res_exp.rmse()

In [ ]:
res.rmse()

In [ ]:
#| exports
@patch
def r2(self: ImputationResult):
    return self.compute_metric(r2_score, "r2")

In [ ]:
res.r2()

In [ ]:
#| export
@patch
def print_metrics(self: ImputationResult):
    
    old = self.metrics_all_data
    
    self.metrics_all_data = True
    all_met = {
    'r2': self.r2(),
    'RMSE': self.rmse()
    }
    
    self.metrics_all_data = False
    met = {**all_met,
    'r2 - Only GAP': self.r2(),
    'RMSE - Only GAP': self.rmse()
    }
    
    self.metrics_all_data = old
    return met

### Prediction plot

In [ ]:
alt.Chart(pd.DataFrame({'a': [1,2,3]})).mark_tick().encode(x = 'a')

In [ ]:
#| exports
def _plot_variable(imp, complete, variable, y_label="", sel=None, properties = {}):
    
    imp = imp[imp.variable == variable]
    sel = sel if sel is not None else alt.selection_interval(bind="scales")
    
    error = alt.Chart(imp).mark_errorband().encode(
        x = "time",    
        y = alt.Y("err_low:Q", title = y_label, scale=alt.Scale(zero=False)),
        y2 = "err_high:Q",
        color=alt.Color("variable",
                        legend = alt.Legend(title=["Line: pred. mean", "area: +/- 2 std", "(variable)"])
                       ),
        tooltip = alt.Tooltip(['std', 'mean'], format=".4")
    ).transform_calculate(
        err_low = "datum.mean - 2 * datum.std",
        err_high = "datum.mean + 2 * datum.std"
    ).properties( **properties)

    pred = alt.Chart(imp).mark_line().encode(
        x = "time",    
        y = alt.Y("mean:Q", title = y_label, scale=alt.Scale(zero=False)),
        color="variable",
    ).add_selection(
        sel
    ).properties(title = variable)

    base_plot = error + pred
    
    if complete is not None:

        complete = complete[complete.variable == variable]
        truth_plt = alt.Chart(complete).mark_point(
            color='black',
            strokeWidth = 1,
            fillOpacity = 1
        ).encode(
            x = alt.X("time", axis=alt.Axis(domain=False, labels = False, ticks=False, title=None)),
            y = alt.Y("value", title = y_label, scale=alt.Scale(zero=False)),
            fill= alt.Fill("is_missing", scale = alt.Scale(range=["#ffffff00", "black"]),
                           legend = alt.Legend(title =["Observed data","(is missing)"])),
            shape = "is_missing",
        )
       
        p = {'width': properties['width']} if properties else {}
        missing = alt.Chart(complete).mark_tick(
            color='black',
        ).encode(
            x = "time",
            color = alt.condition(datum.is_missing, alt.value('black'), alt.value('white'))
        ).add_selection(
            sel
        ).properties(**p)

        base_plot = alt.VConcatChart(vconcat=[(truth_plt + base_plot), missing], spacing=-10)
        
    return base_plot
    

In [ ]:
imp_d = imp.impute(tidy=True, add_time=True)

_plot_variable(imp_d, None, "x1")

In [ ]:
#| exports
@patch()
def plot_pred(
    self: ImputationResult,
    n_cols: int = 2,
    bind_interaction: bool =True, # Whether the sub-plots for each variable should be connected for zooming/panning
    properties:dict = {} # additional properties (eg. size) for altair plot
):
    "Plot the prediction for each variable"
   
    plot_list = [alt.hconcat() for _ in range(0, self.data_imputed.shape[0], n_cols)]
    selection_scale = alt.selection_interval(bind="scales", encodings=['x']) if bind_interaction else None
    for idx, variable in enumerate(pd.unique(self.data_imputed.variable)):
        plot_list[idx // n_cols] |= _plot_variable(self.data_imputed,
                                                   self.data_complete,
                                                   variable,
                                                   y_label = f"{variable} [{self.units[variable]}]" if self.units is not None else variable,
                                                   sel = selection_scale, properties=properties)
    
    plot = alt.vconcat(*plot_list)
    
    return plot

In [ ]:
res.plot_pred()

In [ ]:
res.plot_pred(bind_interaction=False)

    The code is running correctly and as expected around the missing data point the error is band is wider

### Display results

show the prediction plot, metrics and model parameters in one convinient view

In [ ]:
#| export
from IPython.display import HTML

from ipywidgets import HBox, VBox, interact, widgets
from ipywidgets.widgets import Output

In [ ]:
#| export
def _style_df(df):
    """style dataframe for better printing """
    return df.style.hide(axis="index").format(precision = 4)

def _display_as_row(dfs: dict[str, pd.DataFrame], title="", styler=_style_df):
    """display multiple dataframes in the same row"""
    out = []
    for df_title, df in dfs.items():
        df_html = _style_df(df).to_html()
        out.append(f"<div> <p style='font-size: 1.3rem;'>{df_title}</p> {df_html} </div>")
    out = f"<div style=\"display: flex; column-gap: 20px; flex-wrap: wrap;\" class='table table-striped table-sm'> {''.join(out)}</div>"
    display(HTML(f"<p style='font-size: 1.5rem; font-decoration: bold'>{title}<p>" + "".join(out)))

In [ ]:
a = HTML(pd.DataFrame([1,2]).to_html(notebook=True))

In [ ]:
_display_as_row({"test": pd.DataFrame([1,2])}, "hello")

In [ ]:
_display_as_row({f"test{i}": pd.DataFrame([1,2]) for i in range(10)})

In [ ]:
#| exports
@patch 
def display_results(self: ImputationResult, plot_args={}):
    
    plot_args = {'properties': {'height': 200 , 'width': 350}, **plot_args} # set default plot size
    plot = self.plot_pred(**plot_args)
    
    display(plot)    
    _display_as_row(self.print_metrics(), "Metrics")
    _display_as_row(self.model_info, "Model Info")   

In [ ]:
res.model_info

In [ ]:
res.display_results()

## Export 

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()